In [1]:
import csv
from lingpy import *
import itertools
import os
import pickle as pk
from lingpy.settings import rcParams
from itertools import groupby
from asjp import ipa2asjp, asjp2ipa, tokenise
import fileinput
import pickle
import re 
import numpy as np
import pandas as pd
import time
import math

In [32]:
data_set1 = pd.read_csv('/home/bella/Dropbox/Master-Thesis/MA_Thesis_Program/Processed_Data/celtic.csv', 'r',delimiter=',',na_filter=True,header=0)

In [33]:
data_set1

,BRETON,CORNISH,SCOTTISH_GAELIC,IRISH_GAELIC,MANX,OLD_IRISH,PROTO_CELTIC,WELSH
0,bre_mE,cor_mi,gla_mi,gle_mye,glv_me,sga_me,pcl_me,cym_mi
1,bre_tE,cor_ti,gla_hu,gle_tu,glv_Siu,sga_tu,pcl_tu,cym_ti
2,bre_ni,cor_ni,gla_Si5,gle_midy,glv_Sin,sga_sni,pcl_snis,cym_ni
3,bre_inan,cor_onEn,gla_unx,gle_in,glv_un,sga_oen,pcl_oino,cym_3n
4,bre_dau,cor_diu,gla_ta,gle_xa,glv_de,sga_dau,pcl_dwei,cym_da3
5,bre_dEn,cor_den,gla_tu5E,gle_di53,glv_du53,sga_duine,pcl_donyo,cym_d3n
6,bre_pEsk,cor_pisk,gla_i3sk,gle_i3sk,glv_yis,sga_iask,pcl_peisko,cym_p3sgod3n
7,bre_ki,cor_ki,gla_khu,gle_ku,glv_ma83,sga_ku,pcl_kwu,cym_ki
8,bre_lau,cor_lowen,gla_myi3l,gle_sor,glv_mil,sga_mil,pcl_lous,cym_L33En
9,bre_gwEzEn,cor_gwi8en,gla_khrxuv,gle_kran,glv_bily3,sga_kran,pcl_kwrano,cym_prEn


In [34]:
data_set1 = data_set1.replace(np.nan, 'NaN', regex=True)

In [35]:
data_set = []
for row in data_set1.iterrows():
    index, data = row
    data_set.append(data.tolist())

In [36]:
data_set1.isna().sum()

BRETON             0
CORNISH            0
SCOTTISH_GAELIC    0
IRISH_GAELIC       0
MANX               0
OLD_IRISH          0
PROTO_CELTIC       0
WELSH              0
dtype: int64

In [37]:
seqA = []
seqB = []
for i in range(len(data_set)):
    for a,b in itertools.combinations(data_set[i],2):
        seqA.append(a)
        seqB.append(b)
print('compare', a, 'and', b)

compare pcl_anman and cym_Enu


In [13]:
scorer = {}
listoflist = []
with open("pmiScores.csv", "r") as infile:
    for line in infile:
        listoflist.append(line.rstrip("\n").split(','))
        

for i in range(1,len(listoflist)):
    for j in range(1, len(listoflist[0])):
        scorer[(listoflist[0][i], listoflist[j][0])] = float(listoflist[i][j])
        #Scorer is done
#scorer

In [38]:
start = time.time()
final_scores = []
for i in range(len(seqA)):
    if seqA[i][4:] == "NaN" or seqB[i][4:] == "NaN":
        final_scores.append("NaN")
        print(i, seqA[i], seqB[i],final_scores[i],file=open("final_results_celtic.csv", "a"))

        
    else:
        almA, almB, sim = nw_align(seqA[i][4:], seqB[i][4:], scorer=scorer)
        final_scores.append(sim)
        #print(i, seqA[i], seqB[i],final_scores[i])

        print(i, seqA[i], seqB[i],final_scores[i],file=open("final_results_celtic.csv", "a"))
        
end = time.time()
print(end - start)

0.26883697509765625
